In [13]:
import random

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

from mmoe import MMoE

In [14]:
input_layer = Input(shape=(1,))

# Set up MMoE layer
mmoe_layers = MMoE(
    units=4,
    num_experts=8,
    num_tasks=2
)(input_layer)

output_info = [(2, 'click'), (2, 'deep_read')]
output_layers = []

# Build tower layer from MMoE layer
for index, task_layer in enumerate(mmoe_layers):
    tower_layer = Dense(
        units=8,
        activation='relu',
        kernel_initializer=VarianceScaling())(task_layer)
    output_layer = Dense(
        units=output_info[index][0],
        name=output_info[index][1],
        activation='softmax',
        kernel_initializer=VarianceScaling())(tower_layer)
    output_layers.append(output_layer)
    
model = Model(inputs=[input_layer], outputs=output_layers)
adam_optimizer = Adam()
model.compile(
    loss={'click': 'binary_crossentropy', 'deep_read': 'binary_crossentropy'},
    optimizer=adam_optimizer,
    metrics=['accuracy']
)

# Print out model architecture summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
m_mo_e_6 (MMoE)                 [(None, 4), (None, 4 96          input_8[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 8)            40          m_mo_e_6[0][0]                   
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 8)            40          m_mo_e_6[0][1]                   
____________________________________________________________________________________________

In [15]:
train_data = {
    'user_status':[0,1,2]
}
train_frame = pd.DataFrame(train_data)

lable_data = {
    'click':[0,1,1],
    'deep_read':[0,0,1],
    'deep_read':[0,0,0]
}
label_frame = pd.DataFrame(lable_data)

validation_data = {
    'user_status':[0,1,2]
}
validation_label = pd.DataFrame(train_data)

lable_data = {
    'click':[0,1,1],
    'deep_read':[0,0,1],
    'deep_read':[0,0,0]
}
label_frame = pd.DataFrame(lable_data)

# Train the model
model.fit(
    x=train_frame,
    y=label_frame,
    validation_data=(validation_data, validation_label),
    callbacks=[
        ROCCallback(
            training_data=(train_data, train_label),
            validation_data=(validation_data, validation_label),
            test_data=(test_data, test_label)
        )
    ],
    epochs=1
)

NameError: name 'ROCCallback' is not defined